# Imports and Constants

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

import sys
sys.path.append("..")
from backend.func import DemoProcessing
from backend.constants import DEMOS_DIR, STATUS_MAP, REASON_MAP
from demoparser2 import DemoParser

import os

In [90]:
# Define the path to the 'demos' folder
DEMOS_FOLDER = os.path.join(os.path.dirname(''), "..", "demos")

ASSETS_FOLDER = os.path.join(os.path.dirname(''), "..", "assets")
MAPS_BACKGROUND_FOLDER = os.path.join(ASSETS_FOLDER, "maps_background")

# Load all demos paths
demos_paths = [os.path.join(DEMOS_FOLDER, f) for f in os.listdir(DEMOS_FOLDER) if f.endswith('.dem')]
print(f"Found {len(demos_paths)} demo files. Listing first 5: {demos_paths[:5]}")

# Load all map background images paths
maps_background_paths = {f.split('.')[0]: os.path.join(MAPS_BACKGROUND_FOLDER, f) for f in os.listdir(MAPS_BACKGROUND_FOLDER) if f.endswith('.png')}

Found 7 demo files. Listing first 5: ['../demos/nrg-vs-imperial-m2-train.dem', '../demos/imperial-vs-flyquest-inferno.dem', '../demos/imperial-vs-the-huns-m2-mirage.dem', '../demos/imperial-vs-the-huns-m1-nuke.dem', '../demos/fnatic-vs-imperial-train.dem']


# Parse Demos

In [92]:
# Parse a demo file
selected_demo_index = 0
print(f"Parsing demo: {demos_paths[selected_demo_index]}")
parser = DemoParser(demo_path=demos_paths[selected_demo_index])
ticks_df = parser.parse_ticks(wanted_props=['tick', 'X', 'Y', 'health', 'weapon_name', 'is_freeze_period', 'is_warmup_period','team_name', 'round_win_status', 'round_win_reason', 'bomb_planted', 'round_start_time',
'round_end_time', 'is_bomb_planted', 'game_time', 'total_rounds_played'])

parser.parse_header()

Parsing demo: ../demos/nrg-vs-imperial-m2-train.dem


{'patch_version': '14126',
 'demo_file_stamp': 'PBDEMS2\x00',
 'client_name': 'SourceTV Demo',
 'allow_clientside_entities': 'true',
 'demo_version_guid': '8e9d71ab-04a1-4c01-bb61-acfede27c046',
 'allow_clientside_particles': 'true',
 'server_name': 'StarLadder Major Budapest 2025 | A',
 'addons': '',
 'demo_version_name': 'valve_demo_2',
 'map_name': 'de_train',
 'game_directory': '/home/cs2server/cs-gs1/serverfiles/game/csgo',
 'fullpackets_version': '2'}

In [93]:
ticks_df.head()

,is_freeze_period,is_warmup_period,round_start_time,total_rounds_played,is_bomb_planted,round_win_status,round_win_reason,health,team_name,X,Y,weapon_name,game_time,tick,steamid,name
0,True,False,661.21875,0,False,0,0,100,TERRORIST,-1850.0,1256.000000,Butterfly Knife,642.78125,0,76561197975129851,Sonic
1,True,False,661.21875,0,False,0,0,100,CT,1600.0,-1440.000000,P2000,642.78125,0,76561198365118288,try
2,True,False,661.21875,0,False,0,0,100,CT,1456.0,-1328.000000,USP-S,642.78125,0,76561198067763828,chelo
3,True,False,661.21875,0,False,0,0,100,TERRORIST,-2000.0,1434.233154,Glock-18,642.78125,0,76561198060483793,br0
4,True,False,661.21875,0,False,0,0,100,CT,1462.0,-1226.000000,USP-S,642.78125,0,76561197996370184,VINI


In [94]:
# Capture full tick stream (including freeze/warmup) to evaluate round winners
ticks_all = ticks_df.copy()

def last_non_zero(series):
    non_zero = series[series != 0]
    return int(non_zero.iloc[-1]) if not non_zero.empty else 0

ticks_all['round'] = ticks_all['total_rounds_played'].astype('int32', errors='ignore') + 1
round_results = (
    ticks_all.groupby('round')
    .agg(round_winner=('round_win_status', last_non_zero),
         round_reason=('round_win_reason', last_non_zero))
    .reset_index()
)
round_results = round_results[round_results['round_winner'] != 0]

round_results

,round,round_winner,round_reason
1,2,3,8
2,3,3,7
3,4,3,8
4,5,3,8
5,6,3,8
6,7,3,8
7,8,3,8
8,9,3,8
9,10,2,9
10,11,3,8


In [95]:
# Remove Freeze and Warmup time
ticks_df = ticks_df[(ticks_df['is_freeze_period'] == False) & (ticks_df['is_warmup_period'] == False)]
ticks_df.drop(columns=['is_freeze_period', 'is_warmup_period'], inplace=True)

# Use total_rounds_played as the round index
ticks_df['round'] = ticks_df['total_rounds_played'].astype('int32', errors='ignore') + 1

# Join with round-level outcomes computed before filtering
ticks_df = ticks_df.merge(round_results, on='round', how='left')

# Drop non-official rounds (e.g., warmup)
ticks_df = ticks_df.dropna(subset=['round_winner']).copy()
ticks_df['round_winner'] = ticks_df['round_winner'].astype('int32')
ticks_df['round_reason'] = ticks_df['round_reason'].fillna(0).astype('int32')

# Replace per-tick status/reason with aggregated round results
ticks_df['round_win_status'] = ticks_df['round_winner']
ticks_df['round_win_reason'] = ticks_df['round_reason']

# Renumber rounds to be contiguous starting at 1
official_rounds = sorted(round_results['round'].unique())
round_mapping = {r: idx + 1 for idx, r in enumerate(official_rounds)}
ticks_df['round'] = ticks_df['round'].map(round_mapping)
round_results = round_results.assign(round=round_results['round'].map(round_mapping))

# Clean helper columns
ticks_df.drop(columns=['round_winner', 'round_reason'], inplace=True)

In [69]:
# Seconds elapsed in round
ticks_df['seconds_elapsed_in_round'] = (ticks_df['game_time'] - ticks_df['round_start_time']).clip(lower=0)
ticks_df.drop(columns=['round_start_time', 'game_time'], inplace=True)

In [70]:
ticks_df.sort_values(by=['round', 'tick'], inplace=True)

In [81]:
ticks_df

,total_rounds_played,is_bomb_planted,round_win_status,round_win_reason,health,team_name,X,Y,weapon_name,tick,steamid,name,round,seconds_elapsed_in_round
11800,0,False,0,0,100,TERRORIST,-1850.000000,1256.000000,Butterfly Knife,1180,76561197975129851,Sonic,1,0.000000
11801,0,False,0,0,100,CT,1600.000000,-1440.000000,Karambit,1180,76561198365118288,try,1,0.000000
11802,0,False,0,0,100,CT,1456.000000,-1328.000000,M9 Bayonet,1180,76561198067763828,chelo,1,0.000000
11803,0,False,0,0,100,TERRORIST,-2000.000000,1434.233154,Karambit,1180,76561198060483793,br0,1,0.000000
11804,0,False,0,0,100,CT,1462.000000,-1226.000000,Classic Knife,1180,76561197996370184,VINI,1,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1437175,17,True,0,0,0,TERRORIST,-430.822266,1355.440186,None,143766,76561198118646644,skullz,18,85.453125
1437176,17,True,0,0,0,CT,-1012.492615,-1484.466431,None,143766,76561197980158081,daps,18,85.453125
1437177,17,True,0,0,11,TERRORIST,-143.969116,-1529.094849,AK-47,143766,76561198800209567,noway,18,85.453125
1437178,17,True,0,0,0,CT,1050.279297,-926.775269,None,143766,76561198353869335,jeorge,18,85.453125


In [ ]:
# Get tick for start of each round
round_start_ticks = ticks_df.groupby('round')['tick'].min().reset_index()
round_start_ticks.rename(columns={'tick': 'round_start_tick'}, inplace=True)

round_outcomes = ticks_df.groupby('round')['round_win_status'].last().reset_index(name='round_outcome')
round_start_ticks = round_start_ticks.merge(round_outcomes, on='round', how='left')

In [79]:
round_start_ticks.tail()

,round,round_start_tick,round_outcome
13,15,104053,0
14,16,109111,0
15,17,118071,0
16,18,125536,0
17,19,136697,0


In [73]:
# Get all ticks from ticks_df from the first round
for i in range(ticks_df['round'].min(), ticks_df['round'].max() + 1):
    print(f"Unique round_win_status in round {i}: ", end="")
    print(ticks_df.loc[ticks_df['round'] == i, :]['round_win_status'].unique())

Unique round_win_status in round 1: [0]
Unique round_win_status in round 2: [3 0]
Unique round_win_status in round 3: [3 0]
Unique round_win_status in round 4: [3 0]
Unique round_win_status in round 5: [3 0]
Unique round_win_status in round 6: [3 0]
Unique round_win_status in round 7: [3 0]
Unique round_win_status in round 8: [3 0]
Unique round_win_status in round 9: [3 0]
Unique round_win_status in round 10: [2 0]
Unique round_win_status in round 11: [3 0]
Unique round_win_status in round 12: [3 0]
Unique round_win_status in round 13: [2 0]
Unique round_win_status in round 14: [3 0]
Unique round_win_status in round 15: [3 0]
Unique round_win_status in round 16: [3 0]
Unique round_win_status in round 17: [2 0]
Unique round_win_status in round 18: [2 0]


In [77]:
round_start_ticks

,round,round_start_tick,round_outcome
0,2,1180,0
1,3,4638,0
2,4,9697,0
3,5,16197,0
4,6,24694,0
5,7,32661,0
6,8,41533,0
7,9,52568,0
8,10,58903,0
9,11,67795,0


In [ ]:
# Round outcome
ticks_df["round_win_status_label"] = ticks_df["round_win_status"].map(STATUS_MAP).fillna("unknown")
ticks_df["round_win_reason_label"] = ticks_df["round_win_reason"].map(REASON_MAP).fillna("unknown")
ticks_df["round_win_status"] = ticks_df["round_win_status"].astype('int32').astype('category')

KeyError: 'round'

In [29]:
ticks_df

,is_bomb_planted,round_win_status,health,team_name,X,Y,weapon_name,tick,steamid,name,seconds_elapsed_in_round,round_win_status_label,round_win_reason_label
11800,False,0,100,TERRORIST,-1850.000000,1256.000000,Butterfly Knife,1180,76561197975129851,Sonic,0.000000,none,none
11801,False,0,100,CT,1600.000000,-1440.000000,Karambit,1180,76561198365118288,try,0.000000,none,none
11802,False,0,100,CT,1456.000000,-1328.000000,M9 Bayonet,1180,76561198067763828,chelo,0.000000,none,none
11803,False,0,100,TERRORIST,-2000.000000,1434.233154,Karambit,1180,76561198060483793,br0,0.000000,none,none
11804,False,0,100,CT,1462.000000,-1226.000000,Classic Knife,1180,76561197996370184,VINI,0.000000,none,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1437175,True,0,0,TERRORIST,-430.822266,1355.440186,None,143766,76561198118646644,skullz,85.453125,none,none
1437176,True,0,0,CT,-1012.492615,-1484.466431,None,143766,76561197980158081,daps,85.453125,none,none
1437177,True,0,11,TERRORIST,-143.969116,-1529.094849,AK-47,143766,76561198800209567,noway,85.453125,none,none
1437178,True,0,0,CT,1050.279297,-926.775269,None,143766,76561198353869335,jeorge,85.453125,none,none


# Train

In [ ]:
X = processor.ticks_df[["t_alive", "ct_alive", "seconds_elapsed_in_round", "is_bomb_planted", "steamid", "team_name", "weapon_name"]]
y = processor.ticks_df["target"]

In [6]:
X

,t_alive,ct_alive,seconds_elapsed_in_round,is_bomb_planted
11800,5,5,0.000000,False
11801,5,5,0.000000,False
11802,5,5,0.000000,False
11803,5,5,0.000000,False
11804,5,5,0.000000,False
...,...,...,...,...
1437175,5,5,85.453125,True
1437176,5,5,85.453125,True
1437177,5,5,85.453125,True
1437178,5,5,85.453125,True


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = GradientBoostingClassifier(verbose=1)
model.fit(X_train, y_train)

      Iter       Train Loss   Remaining Time 
         1           0.2314            4.03m
         1           0.2314            4.03m
         2           0.2196            3.30m
         2           0.2196            3.30m
         3           0.2104            3.12m
         3           0.2104            3.12m
         4           0.2041            2.99m
         4           0.2041            2.99m
         5           0.1982            2.90m
         5           0.1982            2.90m
         6           0.1940            2.83m
         6           0.1940            2.83m
         7           0.1899            2.82m
         7           0.1899            2.82m
         8           0.1868            2.75m
         8           0.1868            2.75m
         9           0.1842            2.76m
         9           0.1842            2.76m
        10           0.1819            2.73m
        10           0.1819            2.73m
        20           0.1671            2.36m
        2

,loss,'log_loss'
,learning_rate,0.1
,n_estimators,100
,subsample,1.0
,criterion,'friedman_mse'
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_depth,3
,min_impurity_decrease,0.0
,init,None


# Prediction

In [ ]:
prob = model.predict_proba(X_test)
pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, pred))
print("Probabilities example:", prob[:10])

Accuracy: 0.9443569706865718
Probabilities example: [[9.99504805e-01 8.15693597e-06 4.87037870e-04]
 [7.81388365e-01 1.09444167e-01 1.09167468e-01]
 [5.06669048e-01 4.98250381e-04 4.92832701e-01]
 [9.99504805e-01 8.15693597e-06 4.87037870e-04]
 [9.99504805e-01 8.15693597e-06 4.87037870e-04]
 [9.99504805e-01 8.15693597e-06 4.87037870e-04]
 [9.99504805e-01 8.15693597e-06 4.87037870e-04]
 [9.98684676e-01 2.07812532e-05 1.29454255e-03]
 [9.98684676e-01 2.07812532e-05 1.29454255e-03]
 [8.49033327e-01 7.64544617e-02 7.45122110e-02]]
